In [2]:
import tensorflow as tf
from tensorflow.python.client import device_lib
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3435858766010733562
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 4834983936
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6758879376051890711
 physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1060, pci bus id: 0000:01:00.0, compute capability: 6.1"
 xla_global_id: 416903419]

In [4]:
import os
import cv2
import numpy as np
import torch
import glob
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import pathlib
from tensorflow.keras.utils import img_to_array
from sklearn.model_selection import train_test_split

In [5]:
device = torch.device('cuda' if torch.cuda.is_available()else'cpu')

In [6]:
print(device)

cpu


In [7]:
#transforms
transformer=transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],
                        [0.5,0.5,0.5])
]
)

In [8]:
listImageFilePaths = []
for dirname, _, filenames in os.walk('combined_faces/'):
    for filename in filenames:
        listImageFilePaths.append( os.path.join(dirname, filename))
        
print('Total images ', len(listImageFilePaths))

Total images  33486


In [9]:
squeezenet_age = 'model/agegender_age101_squeezenet.hdf5'

In [10]:
listImageFilePaths[:2]

['combined_faces/100_1.jpg', 'combined_faces/100_10.jpg']

In [11]:
Image_dimension = (100, 100, 3)

In [12]:
data = []
labels = []

In [13]:
def displayImage(imagePath):
    image = mping.imread(imagePath)
    plt.imshow(image)
    print('Image path', imagePath)
    plt.show()

In [14]:
for img in listImageFilePaths:
    image1 = cv2.imread(img)
    image = img_to_array(image1)
    data.append(image) 
    age_value = img.split("/")[-1]
    age_value = age_value.split("_")[0]
    labels.append(age_value)

In [15]:
labels[550:555]

['12', '12', '12', '12', '12']

In [16]:
data[:3]

[array([[[32., 41., 51.],
         [31., 40., 50.],
         [29., 38., 48.],
         ...,
         [67., 79., 85.],
         [64., 74., 81.],
         [62., 72., 79.]],
 
        [[32., 41., 51.],
         [31., 40., 50.],
         [29., 38., 48.],
         ...,
         [68., 80., 86.],
         [65., 75., 82.],
         [62., 72., 79.]],
 
        [[31., 40., 50.],
         [29., 38., 48.],
         [28., 37., 47.],
         ...,
         [69., 81., 87.],
         [68., 78., 85.],
         [66., 76., 83.]],
 
        ...,
 
        [[63., 74., 78.],
         [67., 78., 82.],
         [71., 82., 86.],
         ...,
         [59., 74., 83.],
         [55., 70., 79.],
         [53., 68., 77.]],
 
        [[64., 75., 79.],
         [67., 78., 82.],
         [72., 83., 87.],
         ...,
         [61., 76., 85.],
         [58., 73., 82.],
         [55., 70., 79.]],
 
        [[63., 74., 78.],
         [66., 77., 81.],
         [68., 79., 83.],
         ...,
         [63., 78., 87.],
  

In [17]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [18]:
print(y_train[4])

5


In [19]:
#DataLoader
train_loader = DataLoader(
    torchvision.datasets.ImageFolder(x_train, transform=transformer),
    batch_size=256, shuffle=True
)
test_loader=Dataloader(
    torchvision.datasets.ImageFolder(x_test, transform=transformer),
    batch_size=256, shuffle=True
)

TypeError: scandir: path should be string, bytes, os.PathLike or None, not list

In [ ]:
parameters = {'n_estimators':[4, 6, 9],
              'max_features':['log2', 'sqrt', 'auto'],
              'criterion': ['entropy', 'gini'],
              'max_depth': [2, 3, 5, 10], 
              'min_samples_split': [2, 3, 5],
              'min_samples_leaf': [1,5,8]
             }

In [ ]:
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV

acc_scorer = make_scorer(accuracy_score)

grid_obj = GridSearchCV(squeezenet_age, parameters, scoring=acc_scorer)
grid_obj = grid_obj.fit(x_train, y_train)

squeezenet_age = grid_obj.best_estimator_
squeezenet_age.fit(x_train, y_train)